In [1]:
# Imports
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [2]:
# Load SP500 stocks data
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url, verify=False)  # Set verify to False to ignore SSL certificate verification

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')
sp500 = pd.read_html(str(soup.find_all('table')[0]))[0]

#replace some symbols to use in yfinance
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

#create a list with SP500 tickers
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list = ['AAPL', 'GOOGL', 'META', 'MSFT', 'TSLA'] # temporary to not run all the sp500 tickers

#select stard and end date
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8) # remove five years from the end_date

df = yf.download(tickers=symbols_list,
                start=start_date,
                end=end_date)
df

[*********************100%***********************]  5 of 5 completed


Adj Close                                      \
                                 AAPL       GOOGL        META        MSFT   
Date                                                                        
2015-09-29 00:00:00-04:00   24.716064   31.130501   86.669998   39.194000   
2015-09-30 00:00:00-04:00   24.997084   31.918501   89.900002   39.933853   
2015-10-01 00:00:00-04:00   24.833914   32.099998   90.949997   40.249638   
2015-10-02 00:00:00-04:00   25.015217   32.849499   92.070000   41.115807   
2015-10-05 00:00:00-04:00   25.105867   33.584000   94.010002   42.072197   
...                               ...         ...         ...         ...   
2023-09-20 00:00:00-04:00  175.259109  133.740005  299.670013  320.769989   
2023-09-21 00:00:00-04:00  173.701157  130.440002  295.730011  319.529999   
2023-09-22 00:00:00-04:00  174.560013  130.250000  299.079987  317.010010   
2023-09-25 00:00:00-04:00  175.848328  131.110001  300.829987  317.540009   
2023-09-26 00:00:00-04:00  171.733749  128.570007  298.959991  312.140015   

                                            Close                          \
                                 TSLA        AAPL       GOOGL        META   
Date                                                                        
2015-09-29 00:00:00-04:00   16.443333   27.264999   31.130501   86.669998   
2015-09-30 00:00:00-04:00   16.559999   27.575001   31.918501   89.900002   
2015-10-01 00:00:00-04:00   15.992000   27.395000   32.099998   90.949997   
2015-10-02 00:00:00-04:00   16.504667   27.594999   32.849499   92.070000   
2015-10-05 00:00:00-04:00   16.410000   27.695000   33.584000   94.010002   
...                               ...         ...         ...         ...   
2023-09-20 00:00:00-04:00  262.589996  175.490005  133.740005  299.670013   
2023-09-21 00:00:00-04:00  255.699997  173.929993  130.440002  295.730011   
2023-09-22 00:00:00-04:00  244.880005  174.789993  130.250000  299.079987   
2023-09-25 00:00:00-04:00  246.990005  176.080002  131.110001  300.829987   
2023-09-26 00:00:00-04:00  244.119995  171.960007  128.570007  298.959991   

                                                   ...        Open  \
                                 MSFT        TSLA  ...        AAPL   
Date                                               ...               
2015-09-29 00:00:00-04:00   43.439999   16.443333  ...   28.207500   
2015-09-30 00:00:00-04:00   44.259998   16.559999  ...   27.542500   
2015-10-01 00:00:00-04:00   44.610001   15.992000  ...   27.267500   
2015-10-02 00:00:00-04:00   45.570000   16.504667  ...   27.002501   
2015-10-05 00:00:00-04:00   46.630001   16.410000  ...   27.469999   
...                               ...         ...  ...         ...   
2023-09-20 00:00:00-04:00  320.769989  262.589996  ...  179.259995   
2023-09-21 00:00:00-04:00  319.529999  255.699997  ...  174.550003   
2023-09-22 00:00:00-04:00  317.010010  244.880005  ...  174.669998   
2023-09-25 00:00:00-04:00  317.540009  246.990005  ...  174.199997   
2023-09-26 00:00:00-04:00  312.140015  244.119995  ...  174.820007   

                                                                           \
                                GOOGL        META        MSFT        TSLA   
Date                                                                        
2015-09-29 00:00:00-04:00   31.275000   89.099998   43.369999   16.697332   
2015-09-30 00:00:00-04:00   31.591499   88.440002   43.880001   16.799999   
2015-10-01 00:00:00-04:00   31.896999   90.050003   44.750000   16.500668   
2015-10-02 00:00:00-04:00   31.900499   88.900002   44.270000   15.706667   
2015-10-05 00:00:00-04:00   33.080002   92.940002   45.750000   16.589333   
...                               ...         ...         ...         ...   
2023-09-20 00:00:00-04:00  138.080002  305.049988  329.510010  267.040009   
2023-09-21 00:00:00-04:00  131.440002  295.700012  319.260010  257.850006   
2023-09-22 00:00:00-04:00  130.759995  299

In [3]:
#Organize our indexexaset to be more clear
df = df.stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

adj close       close        high  \
date                      ticker                                       
2015-09-29 00:00:00-04:00 AAPL     24.716064   27.264999   28.377501   
                          GOOGL    31.130501   31.130501   31.733999   
                          META     86.669998   86.669998   90.059998   
                          MSFT     39.194000   43.439999   43.570000   
                          TSLA     16.443333   16.443333   16.982000   
...                                      ...         ...         ...   
2023-09-26 00:00:00-04:00 AAPL    171.733749  171.960007  175.199997   
                          GOOGL   128.570007  128.570007  130.360001   
                          META    298.959991  298.959991  300.299988   
                          MSFT    312.140015  312.140015  315.880005   
                          TSLA    244.119995  244.119995  249.550003   

                                         low        open     volume  
date                      ticker                                     
2015-09-29 00:00:00-04:00 AAPL     26.965000   28.207500  293461600  
                          GOOGL    30.901501   31.275000   42130000  
                          META     85.720001   89.099998   42281800  
                          MSFT     43.049999   43.369999   32763600  
                          TSLA     16.364000   16.697332   55548000  
...                                      ...         ...        ...  
2023-09-26 00:00:00-04:00 AAPL    171.660004  174.820007   64588900  
                          GOOGL   127.220001  129.770004   25718700  
                          META    296.010010  297.660004   19417200  
                          MSFT    310.019989  315.130005   26297600  
                          TSLA    241.660004  242.979996  101993600  

[10060 rows x 6 columns]

### Calculate technical indicators

In [4]:
#Garman_klass Volatility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

#RSI
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
#df.xs('AAPL', level=1)['rsi'].plot() # plot rsi of apple

# BollingerBands
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

#ATR
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                 low=stock_data['low'],
                 close=stock_data['close'],
                 lenght=14)
    return atr.sub(atr.mean()).div(atr.std())


df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

#MACD
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std()) #calutation to normalize


df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

#Calculate dollar volume
df['dollar_volume'] = (df['adj close'] * df['volume']) / 1e6 #divide by 1 milion because milions of shares are trated daily

df

adj close       close        high  \
date                      ticker                                       
2015-09-29 00:00:00-04:00 AAPL     24.716064   27.264999   28.377501   
                          GOOGL    31.130501   31.130501   31.733999   
                          META     86.669998   86.669998   90.059998   
                          MSFT     39.194000   43.439999   43.570000   
                          TSLA     16.443333   16.443333   16.982000   
...                                      ...         ...         ...   
2023-09-26 00:00:00-04:00 AAPL    171.733749  171.960007  175.199997   
                          GOOGL   128.570007  128.570007  130.360001   
                          META    298.959991  298.959991  300.299988   
                          MSFT    312.140015  312.140015  315.880005   
                          TSLA    244.119995  244.119995  249.550003   

                                         low        open     volume  \
date                      ticker                                      
2015-09-29 00:00:00-04:00 AAPL     26.965000   28.207500  293461600   
                          GOOGL    30.901501   31.275000   42130000   
                          META     85.720001   89.099998   42281800   
                          MSFT     43.049999   43.369999   32763600   
                          TSLA     16.364000   16.697332   55548000   
...                                      ...         ...        ...   
2023-09-26 00:00:00-04:00 AAPL    171.660004  174.820007   64588900   
                          GOOGL   127.220001  129.770004   25718700   
                          META    296.010010  297.660004   19417200   
                          MSFT    310.019989  315.130005   26297600   
                          TSLA    241.660004  242.979996  101993600   

                                  garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2015-09-29 00:00:00-04:00 AAPL           -0.005441        NaN       NaN   
                          GOOGL           0.000345        NaN       NaN   
                          META            0.000924        NaN       NaN   
                          MSFT           -0.003888        NaN       NaN   
                          TSLA            0.000596        NaN       NaN   
...                                            ...        ...       ...   
2023-09-26 00:00:00-04:00 AAPL            0.000086  40.441250  5.133614   
                          GOOGL           0.000264  43.997716  4.871573   
                          META            0.000096  50.262069  5.681562   
                          MSFT            0.000140  38.358054  5.753026   
                          TSLA            0.000508  45.955272  5.480648   

                                    bb_mid   bb_high       atr      macd  \
date                      ticker                                           
2015-09-29 00:00:00-04:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-09-26 00:00:00-04:00 AAPL    5.193242  5.252870  0.967273 -1.417050   
                          GOOGL   4.912076  4.952579  0.930630 -0.192760   
                          META    5.708585  5.735609  0.961079 -0.040023   
                          MSFT    5.796374  5.839722  0.894571 -1.317375   
                          TSLA    5.558842  5.637036  0.824771 -0.027720   

                                  dollar_volume  
date                      ticker                 
2015-09-29 00:00:00-04:00 AAPL      7253.215820  
                          GOOGL     1311.527998  
         

### Aggregate to monthly level and filter mos liquid stock

In [5]:
df.unstack('ticker')['dollar_volume'].resample('M').mean() #agregate by month

ticker,AAPL,GOOGL,META,MSFT,TSLA
date,,,,,
2015-09-30 00:00:00-04:00,6949.870195,1500.870148,3458.082837,1340.090063,1068.953422
2015-10-31 00:00:00-04:00,5210.462014,1584.213103,2526.490236,1748.174716,1017.910952
2015-11-30 00:00:00-05:00,4034.583584,1426.139264,2901.607961,1615.109580,868.514543
2015-12-31 00:00:00-05:00,4258.527209,1527.790769,2106.832258,1808.617125,627.100746
2016-01-31 00:00:00-05:00,5981.206682,2015.825533,4172.506229,2326.067193,855.033458
...,...,...,...,...,...
2023-05-31 00:00:00-04:00,9977.793395,4357.569070,5400.470759,8631.874445,21573.416958
2023-06-30 00:00:00-04:00,11341.862337,3830.668431,6367.596815,8737.577636,40338.857362
2023-07-31 00:00:00-04:00,9556.823331,4371.732062,9483.127463,11434.356688,32726.593194


In [6]:
#Create dict with indicators columns
indicator_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume',
                                                              'open', 'close', 'high',
                                                              'low']]
indicator_cols

['adj close',
 'garman_klass_vol',
 'rsi',
 'bb_low',
 'bb_mid',
 'bb_high',
 'atr',
 'macd']

In [7]:
df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume')

dollar_volume
date                      ticker               
2015-09-30 00:00:00-04:00 AAPL      6949.870195
                          GOOGL     1500.870148
                          META      3458.082837
                          MSFT      1340.090063
                          TSLA      1068.953422
...                                         ...
2023-09-30 00:00:00-04:00 AAPL     12130.191043
                          GOOGL     3181.391274
                          META      5714.408578
                          MSFT      6804.980194
                          TSLA     31386.906936

[485 rows x 1 columns]

In [8]:
df.unstack()[indicator_cols].resample('M').last().stack('ticker')

adj close  garman_klass_vol        rsi  \
date                      ticker                                            
2015-09-30 00:00:00-04:00 AAPL     24.997084         -0.003307        NaN   
                          GOOGL    31.918501          0.000087        NaN   
                          META     89.900002          0.000151        NaN   
                          MSFT     39.933853         -0.003324        NaN   
                          TSLA     16.559999          0.000747        NaN   
...                                      ...               ...        ...   
2023-09-30 00:00:00-04:00 AAPL    171.733749          0.000086  40.441250   
                          GOOGL   128.570007          0.000264  43.997716   
                          META    298.959991          0.000096  50.262069   
                          MSFT    312.140015          0.000140  38.358054   
                          TSLA    244.119995          0.000508  45.955272   

                                    bb_low    bb_mid   bb_high       atr  \
date                      ticker                                           
2015-09-30 00:00:00-04:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    5.133614  5.193242  5.252870  0.967273   
                          GOOGL   4.871573  4.912076  4.952579  0.930630   
                          META    5.681562  5.708585  5.735609  0.961079   
                          MSFT    5.753026  5.796374  5.839722  0.894571   
                          TSLA    5.480648  5.558842  5.637036  0.824771   

                                      macd  
date                      ticker            
2015-09-30 00:00:00-04:00 AAPL         NaN  
                          GOOGL        NaN  
                          META         NaN  
                          MSFT         NaN  
                          TSLA         NaN  
...                                    ...  
2023-09-30 00:00:00-04:00 AAPL   -1.417050  
                          GOOGL  -0.192760  
                          META   -0.040023  
                          MSFT   -1.317375  
                          TSLA   -0.027720  

[485 rows x 8 columns]

In [9]:
#Combine indicators with dollar_volume mean by month
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume'),
          df.unstack()[indicator_cols].resample('M').last().stack('ticker')],
          axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2015-11-30 00:00:00-05:00 AAPL      4034.583584   26.924877         -0.003119   
                          GOOGL     1426.139264   38.142502          0.000111   
                          META      2901.607961  104.239998          0.000161   
                          MSFT      1615.109580   49.368141         -0.003679   
                          TSLA       868.514543   15.350667          0.000235   
...                                         ...         ...               ...   
2023-09-30 00:00:00-04:00 AAPL     12130.191043  171.733749          0.000086   
                          GOOGL     3181.391274  128.570007          0.000264   
                          META      5714.408578  298.959991          0.000096   
                          MSFT      6804.980194  312.140015          0.000140   
                          TSLA     31386.906936  244.119995          0.000508   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2015-11-30 00:00:00-05:00 AAPL    55.537379  3.284211  3.327527  3.370842   
                          GOOGL   63.168941  3.637679  3.663018  3.688358   
                          META    55.404379  4.638016  4.672391  4.706767   
                          MSFT    67.377488  3.887128  3.907274  3.927421   
                          TSLA    54.242021  2.687592  2.754718  2.821844   
...                                     ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    40.441250  5.133614  5.193242  5.252870   
                          GOOGL   43.997716  4.871573  4.912076  4.952579   
                          META    50.262069  5.681562  5.708585  5.735609   
                          MSFT    38.358054  5.753026  5.796374  5.839722   
                          TSLA    45.955272  5.480648  5.558842  5.637036   

                                       atr      macd  
date                      ticker                      
2015-11-30 00:00:00-05:00 AAPL   -0.967900 -0.142789  
                          GOOGL  -1.033524  0.375191  
                          META   -1.165352  0.202517  
                          MSFT   -1.056035  0.064642  
                          TSLA   -0.809146 -0.114649  
...                                    ...       ...  
2023-09-30 00:00:00-04:00 AAPL    0.967273 -1.417050  
                          GOOGL   0.930630 -0.192760  
                          META    0.961079 -0.040023  
                          MSFT    0.894571 -1.317375  
                          TSLA    0.824771 -0.027720  

[475 rows x 9 columns]

In [10]:
#5 years average of dollar volume
data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank)

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2015-11-30 00:00:00-05:00 AAPL              NaN   26.924877         -0.003119   
                          GOOGL             NaN   38.142502          0.000111   
                          META              NaN  104.239998          0.000161   
                          MSFT              NaN   49.368141         -0.003679   
                          TSLA              NaN   15.350667          0.000235   
...                                         ...         ...               ...   
2023-09-30 00:00:00-04:00 AAPL     11087.551064  171.733749          0.000086   
                          GOOGL     3179.070763  128.570007          0.000264   
                          META      5159.217493  298.959991          0.000096   
                          MSFT      6610.448228  312.140015          0.000140   
                          TSLA     17662.730446  244.119995          0.000508   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2015-11-30 00:00:00-05:00 AAPL    55.537379  3.284211  3.327527  3.370842   
                          GOOGL   63.168941  3.637679  3.663018  3.688358   
                          META    55.404379  4.638016  4.672391  4.706767   
                          MSFT    67.377488  3.887128  3.907274  3.927421   
                          TSLA    54.242021  2.687592  2.754718  2.821844   
...                                     ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    40.441250  5.133614  5.193242  5.252870   
                          GOOGL   43.997716  4.871573  4.912076  4.952579   
                          META    50.262069  5.681562  5.708585  5.735609   
                          MSFT    38.358054  5.753026  5.796374  5.839722   
                          TSLA    45.955272  5.480648  5.558842  5.637036   

                                       atr      macd  \
date                      ticker                       
2015-11-30 00:00:00-05:00 AAPL   -0.967900 -0.142789   
                          GOOGL  -1.033524  0.375191   
                          META   -1.165352  0.202517   
                          MSFT   -1.056035  0.064642   
                          TSLA   -0.809146 -0.114649   
...                                    ...       ...   
2023-09-30 00:00:00-04:00 AAPL    0.967273 -1.417050   
                          GOOGL   0.930630 -0.192760   
                          META    0.961079 -0.040023   
                          MSFT    0.894571 -1.317375   
                          TSLA    0.824771 -0.027720   

                                                                    dollar_vol_rank  
date                      ticker                                                     
2015-11-30 00:00:00-05:00 AAPL    <bound method GroupBy.rank of <pandas.core.gro...  
                          GOOGL   <bound method GroupBy.rank of <pandas.core.gro...  
                          META    <bound method GroupBy.rank of <pandas.core.gro...  
                          MSFT    <bound method GroupBy.rank of <pandas.core.gro...  
                          TSLA    <bound method GroupBy.rank of <pandas.core.gro...  
...                                                                             ...  
2023-09-30 00:00:00-04:00 AAPL    <bound method GroupBy.rank of <pandas.core.gro...  
                          GOOGL   <bound method GroupBy.rank of <pandas.core.gro...  
                          META    <bound method GroupBy.rank of <pandas.core.gro...  
                          MSFT    <bound method GroupBy.rank of <pandas.core.gro...  
                          TSLA    <bound method GroupBy.rank of <pandas.core.gro...  

[475 rows x 10 columns]

In [11]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2020-10-31 00:00:00-04:00 AAPL      6397.417922  106.786552          0.000161   
                          GOOGL     1957.409085   80.805496          0.000803   
                          META      3592.616999  263.109985          0.001466   
                          MSFT      3107.957791  201.013123          0.000209   
                          TSLA      4628.830346  129.346664          0.001753   
...                                         ...         ...               ...   
2023-09-30 00:00:00-04:00 AAPL     11087.551064  171.733749          0.000086   
                          GOOGL     3179.070763  128.570007          0.000264   
                          META      5159.217493  298.959991          0.000096   
                          MSFT      6610.448228  312.140015          0.000140   
                          TSLA     17662.730446  244.119995          0.000508   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2020-10-31 00:00:00-04:00 AAPL    43.770315  4.688665  4.747485  4.806305   
                          GOOGL   56.884334  4.297133  4.361900  4.426668   
                          META    47.938144  5.543932  5.603265  5.662598   
                          MSFT    43.119207  5.303539  5.359551  5.415563   
                          TSLA    44.759949  4.889861  4.963949  5.038038   
...                                     ...       ...       ...       ...   
2023-09-30 00:00:00-04:00 AAPL    40.441250  5.133614  5.193242  5.252870   
                          GOOGL   43.997716  4.871573  4.912076  4.952579   
                          META    50.262069  5.681562  5.708585  5.735609   
                          MSFT    38.358054  5.753026  5.796374  5.839722   
                          TSLA    45.955272  5.480648  5.558842  5.637036   

                                       atr      macd  \
date                      ticker                       
2020-10-31 00:00:00-04:00 AAPL    1.409489 -0.628204   
                          GOOGL   0.776202  0.473605   
                          META    1.756253  0.340548   
                          MSFT    0.702637 -0.743234   
                          TSLA    0.234892 -0.241294   
...                                    ...       ...   
2023-09-30 00:00:00-04:00 AAPL    0.967273 -1.417050   
                          GOOGL   0.930630 -0.192760   
                          META    0.961079 -0.040023   
                          MSFT    0.894571 -1.317375   
                          TSLA    0.824771 -0.027720   

                                                                    dollar_vol_rank  \
date                      ticker                                                      
2020-10-31 00:00:00-04:00 AAPL    <bound method GroupBy.rank of <pandas.core.gro...   
                          GOOGL   <bound method GroupBy.rank of <pandas.core.gro...   
                          META    <bound method GroupBy.rank of <pandas.core.gro...   
                          MSFT    <bound method GroupBy.rank of <pandas.core.gro...   
                          TSLA    <bound method GroupBy.rank of <pandas.core.gro...   
...                                                                             ...   
2023-09-30 00:00:00-04:00 AAPL    <bound method GroupBy.rank of <pandas.core.gro...   
                          GOOGL   <bound method GroupBy.rank of <pandas.core.gro...   
                          META    <bound method GroupBy.rank of <pandas.core.gro...   
                          MSFT    <bound method GroupBy.rank of <pandas.core.gro...   
                          TSLA    <bound method GroupBy.rank of <pandas.core.gro...   

                                  return_1m  return_2m  return_3m  return_6m  \
date                      ticker     

In [12]:
# Download Fama-French factors and calculate betas
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
              'famafrench',
              start='2010')[0].drop('RF', axis=1)

factor_data.index = pd.to_datetime(factor_data.index.to_timestamp())
factor_data

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
2010-01-01,-3.36,0.34,0.43,-1.27,0.46
2010-02-01,3.40,1.51,3.23,-0.27,1.43
2010-03-01,6.31,1.85,2.21,-0.65,1.69
2010-04-01,2.00,4.98,2.89,0.69,1.72
2010-05-01,-7.89,0.04,-2.44,1.30,-0.21
...,...,...,...,...,...
2023-07-01,3.21,2.86,4.11,-0.57,0.57
2023-08-01,-2.39,-3.65,-1.06,3.43,-2.37
2023-09-01,-5.24,-1.80,1.52,1.86,-0.83


In [13]:
#change the fist day of the month to the last and divide by 100 because original is in percent
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'

factor_data

,Mkt-RF,SMB,HML,RMW,CMA
date,,,,,
2010-01-31,-0.0336,0.0034,0.0043,-0.0127,0.0046
2010-02-28,0.0340,0.0151,0.0323,-0.0027,0.0143
2010-03-31,0.0631,0.0185,0.0221,-0.0065,0.0169
2010-04-30,0.0200,0.0498,0.0289,0.0069,0.0172
2010-05-31,-0.0789,0.0004,-0.0244,0.0130,-0.0021
...,...,...,...,...,...
2023-07-31,0.0321,0.0286,0.0411,-0.0057,0.0057
2023-08-31,-0.0239,-0.0365,-0.0106,0.0343,-0.0237
2023-09-30,-0.0524,-0.0180,0.0152,0.0186,-0.0083


In [14]:
#CHange dataset timezone to join
factor_data.index = factor_data.index.tz_localize('EST')

In [15]:
# join factor_data with 1m return
factor_data = factor_data.join(data['return_1m'])
factor_data

Mkt-RF     SMB     HML     RMW     CMA  \
date                      ticker                                           
2020-11-30 05:00:00+00:00 AAPL    0.1247  0.0711  0.0209 -0.0220  0.0140   
                          GOOGL   0.1247  0.0711  0.0209 -0.0220  0.0140   
                          META    0.1247  0.0711  0.0209 -0.0220  0.0140   
                          MSFT    0.1247  0.0711  0.0209 -0.0220  0.0140   
                          TSLA    0.1247  0.0711  0.0209 -0.0220  0.0140   
2020-12-31 05:00:00+00:00 AAPL    0.0463  0.0479 -0.0151 -0.0196 -0.0012   
                          GOOGL   0.0463  0.0479 -0.0151 -0.0196 -0.0012   
                          META    0.0463  0.0479 -0.0151 -0.0196 -0.0012   
                          MSFT    0.0463  0.0479 -0.0151 -0.0196 -0.0012   
                          TSLA    0.0463  0.0479 -0.0151 -0.0196 -0.0012   
2021-01-31 05:00:00+00:00 AAPL   -0.0003  0.0694  0.0301 -0.0378  0.0494   
                          GOOGL  -0.0003  0.0694  0.0301 -0.0378  0.0494   
                          META   -0.0003  0.0694  0.0301 -0.0378  0.0494   
                          MSFT   -0.0003  0.0694  0.0301 -0.0378  0.0494   
                          TSLA   -0.0003  0.0694  0.0301 -0.0378  0.0494   
2021-02-28 05:00:00+00:00 AAPL    0.0278  0.0454  0.0715  0.0039 -0.0194   
                          GOOGL   0.0278  0.0454  0.0715  0.0039 -0.0194   
                          META    0.0278  0.0454  0.0715  0.0039 -0.0194   
                          MSFT    0.0278  0.0454  0.0715  0.0039 -0.0194   
                          TSLA    0.0278  0.0454  0.0715  0.0039 -0.0194   
2021-11-30 05:00:00+00:00 AAPL   -0.0155 -0.0177 -0.0045  0.0720  0.0173   
                          GOOGL  -0.0155 -0.0177 -0.0045  0.0720  0.0173   
                          META   -0.0155 -0.0177 -0.0045  0.0720  0.0173   
                          MSFT   -0.0155 -0.0177 -0.0045  0.0720  0.0173   
                          TSLA   -0.0155 -0.0177 -0.0045  0.0720  0.0173   
2021-12-31 05:00:00+00:00 AAPL    0.0310 -0.0080  0.0326  0.0491  0.0440   
                          GOOGL   0.0310 -0.0080  0.0326  0.0491  0.0440   
                          META    0.0310 -0.0080  0.0326  0.0491  0.0440   
                          MSFT    0.0310 -0.0080  0.0326  0.0491  0.0440   
                          TSLA    0.0310 -0.0080  0.0326  0.0491  0.0440   
2022-01-31 05:00:00+00:00 AAPL   -0.0625 -0.0406  0.1275  0.0084  0.0772   
                          GOOGL  -0.0625 -0.0406  0.1275  0.0084  0.0772   
                          META   -0.0625 -0.0406  0.1275  0.0084  0.0772   
                          MSFT   -0.0625 -0.0406  0.1275  0.0084  0.0772   
                          TSLA   -0.0625 -0.0406  0.1275  0.0084  0.0772   
2022-02-28 05:00:00+00:00 AAPL   -0.0229  0.0292  0.0308 -0.0209  0.0312   
                          GOOGL  -0.0229  0.0292  0.0308 -0.0209  0.0312   
                          META   -0.0229  0.0292  0.0308 -0.0209  0.0312   
                          MSFT   -0.0229  0.0292  0.0308 -0.0209  0.0312   
                          TSLA   -0.0229  0.0292  0.0308 -0.0209  0.0312   
2022-11-30 05:00:00+00:00 AAPL    0.0460 -0.0274  0.0138  0.0638  0.0318   
                          GOOGL   0.0460 -0.0274  0.0138  0.0638  0.0318   
                          META    0.0460 -0.0274  0.0138  0.0638  0.0318   
                          MSFT    0.0460 -0.0274  0.0138  0.0638  0.0318   
                          TSLA    0.0460 -0.0274  0.0138  0.0638  0.0318   
2022-12-31 05:00:00+00:00 AAPL   -0.0641 -0.0015  0.0137  0.0025  0.0420   
                          GOOGL  -0.0641 -0.0015  0.0137  0.0025  0.0420   
                          META   -0.0641 -0.0015  0.0137  0.0025  0.0420   
                          MSFT   -0.0641 -0.0015  0.0137  0.0025  0.0420   
                          TSLA   -0.0641 -0.0015  0.0137  0.0025  0.0420   
2023-01-31 05:00:00+00:00 AAPL    0.0665  0.0441 -0.0401 -0.0244 -0.0447   
      

In [17]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW  \
date                      ticker                                           
2020-11-30 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2020-12-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2021-01-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2021-02-28 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2021-11-30 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2021-12-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2022-01-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2022-02-28 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2022-11-30 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2022-12-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
                          GOOGL        NaN       NaN       NaN       NaN   
                          META         NaN       NaN       NaN       NaN   
                          MSFT         NaN       NaN       NaN       NaN   
                          TSLA         NaN       NaN       NaN       NaN   
2023-01-31 05:00:00+00:00 AAPL         NaN       NaN       NaN       NaN   
        